# Ethereum Address Generator
## Walk-through & Checksum

In [1]:
import codecs
import ecdsa

from Cryptodome.Hash import keccak, SHA256


In [2]:
# KeyCreation ETH image
SHA_seed_str = 'YCIAjPU6k5HTTH5yO8MqnPngA1dSVwxm'
SHA_seed_bytes = bytes(SHA_seed_str, 'utf-8')

# SHA-256 is a case sensitive secure hash algorithm.
h = SHA256.new()
h.update(SHA_seed_bytes)
h.hexdigest()

'1216f219480631acc184622c2912a18a36610e61c087c914b165e6e99d1ec8d6'

In [3]:
# KeyCreation ETH image
private_key = '1216f219480631acc184622c2912a18a36610e61c087c914b165e6e99d1ec8d6'

# https://github.com/Destiner/blocksmith
#private_key = '7077da4a47f6c85a21fe6c6cf1285c0fa06915871744ab1e5a5b741027884d00'

# https://learning.oreilly.com/library/view/mastering-ethereum/9781491971932/ch04.html
#private_key = 'f8f8a2f43c8376ccb0871305060d7b27b0554d2cc72bccf41b2705608452f315'

# https://www.securityevaluators.com/casestudies/ethercombing/
#private_key = '699ee77f6467211cdd03f4012b6fea9377ebd34f107d18c5509ce6ad85113c00'
#private_key = '0000000000000000000000000000000000000000000000000000000000000001'
#private_key = '0000000000000000000000000000000000000000000000000a00000000000000'
#private_key = '4d5db4107d237df6a3d58ee5f70ae63d73d7658d4026f2eefd2f204c81682cb7'

In [4]:
private_key_bytes = codecs.decode(private_key, 'hex')
private_key_bytes

b'\x12\x16\xf2\x19H\x061\xac\xc1\x84b,)\x12\xa1\x8a6a\x0ea\xc0\x87\xc9\x14\xb1e\xe6\xe9\x9d\x1e\xc8\xd6'

In [5]:
# Get ECDSA public key.
# SECP256k1 (ECDSA) is a cryptographic algorithm. Num_in --> Num_out, and is therefore case agnostic.
key = ecdsa.SigningKey.from_string(private_key_bytes, curve=ecdsa.SECP256k1).verifying_key

In [6]:
public_key_bytes = key.to_string()
public_key_bytes

b'e\xe8\x04h*\x16\xc1G\x82\n\xe7\x9bovMg\x14`\xf6\xa5\x12\x93\xd3y\x0b\xc7\xf3\xb0\xd5\xbc.\xdf6\xf0v\xfcS\xb6\xe3\x0b\x0b\xc4\xc7\x05\x9bp\xa81\xc3\xc9HE;\x9b\xfa\xc6\x8fqT\xb0\xbbw\xe0f'

In [7]:
public_key = codecs.encode(public_key_bytes, 'hex')
public_key

b'65e804682a16c147820ae79b6f764d671460f6a51293d3790bc7f3b0d5bc2edf36f076fc53b6e30b0bc4c7059b70a831c3c948453b9bfac68f7154b0bb77e066'

In [8]:
x = public_key[:64]
y = public_key[64:]

print(f'x = {x}')
print(f'y = {y}')

x = b'65e804682a16c147820ae79b6f764d671460f6a51293d3790bc7f3b0d5bc2edf'
y = b'36f076fc53b6e30b0bc4c7059b70a831c3c948453b9bfac68f7154b0bb77e066'


In [9]:
# Serialized EC - Uncompressessed point format:
# 04 + x-coordinate (32 bytes/64 hex) + y-coordinate (32 bytes/64 hex)

In [10]:
# SHA-3 is mentioned throughout Ethereum documents and code,
#  many if not all of those instances actually refer to Keccak-256,
#  not the finalized FIPS-202 SHA-3 standard.
# Check by:

# Keccak256("") = 'c5d2460186f7233c927e7db2dcc703c0e500b653ca82273b7bfad8045d85a470'

# SHA3("") = 'a7ffc6f8bf1ed76651c14756a061d662f580ff4de43b49fa82d80a4b80f8434a'

In [11]:
# keccak-256 is a case sensitive secure hash algorithm.
keccak_hash = keccak.new(digest_bits=256)

keccak_hash.update(public_key_bytes)

keccak_digest = keccak_hash.hexdigest()
keccak_digest

'48419b6f53b3be18c11d2a0608bbe40c7fe2ff78ab07ab11b0b0db9af76022d8'

In [12]:
# Take the last 20 bytes.
wallet_len = 40
wallet_address = '0x' + keccak_digest[-wallet_len:]
wallet_address

'0x08bbe40c7fe2ff78ab07ab11b0b0db9af76022d8'

In [13]:
# Eth EIP-55: Address Checksum.

# In English, convert the address to hex, but if the ith digit is a letter
# (ie. it's one of abcdef) print it in uppercase if the 4*ith bit of the hash
# of the lowercase hexadecimal address is 1 otherwise print it in lowercase.


def checksum_address(wallet_0x_address):
    
    '''Eth EIP-55: Address Checksum Generator'''
    
    address_bytes = bytes.fromhex(wallet_0x_address[2:])

    wallet_address_utf8 = wallet_0x_address[2:].encode('utf-8')

    kh = keccak.new(digest_bits=256)
    kh.update(wallet_address_utf8)

    o = ''
    v = int.from_bytes(kh.digest(), 'big')

    for i, c in enumerate(address_bytes.hex()):
        if c in '0123456789':
           o += c
        else:
           o += c.upper() if (v & (2**(255 - 4*i))) else c.lower()

    return('0x' + o)

In [14]:
print(checksum_address(wallet_address))

0x08bBE40c7fE2Ff78AB07AB11B0B0dB9Af76022d8
